In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from keras import backend as K
import numpy as np
import pickle
import time
# from numba import cuda

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(
        physical_devices[0], True
    )
except:
    pass

In [3]:
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [4]:
# pickle_in = open("X.pickle","rb")
pickle_in = open("X.color.pickle","rb")
X = pickle.load(pickle_in)

# pickle_in = open("y.pickle","rb")
pickle_in = open("y.color.pickle","rb")
y = np.array(pickle.load(pickle_in))

X = X/255.0

In [5]:
dense_layers = [0, 1, 2]
layer_sizes = [32, 64]
conv_layers = [3]

In [6]:
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import gc

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass

    print(gc.collect()) # if it does something you should see a number as output

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.compat.v1.Session(config=config))

In [7]:
def run_tensorflow():
    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
#                 K.clear_session()
                reset_keras()
        
                NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
                tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

                model = Sequential()
                model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
                for l in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))
                    model.add(Dropout(0.2))

                model.add(Dense(1))
                model.add(Activation('sigmoid'))

                model.compile(loss='binary_crossentropy',
                              optimizer='adam',
                              metrics=['accuracy'],
                              )

                model.fit(X, y,
                          batch_size=16,
                          epochs=10,
                          validation_split=0.3,
                          callbacks=[tensorboard])
                                
#                 cuda.select_device(0)            
#                 cuda.close()

In [8]:
run_tensorflow()

0
Epoch 1/10
1092/1092 [==============================] - 8s 5ms/step - loss: 0.6310 - accuracy: 0.6325 - val_loss: 0.5894 - val_accuracy: 0.6768
Epoch 2/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.5161 - accuracy: 0.7449 - val_loss: 0.4767 - val_accuracy: 0.7727
Epoch 3/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.4600 - accuracy: 0.7871 - val_loss: 0.4692 - val_accuracy: 0.7791
Epoch 4/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.4197 - accuracy: 0.8075 - val_loss: 0.4419 - val_accuracy: 0.7914
Epoch 5/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.3922 - accuracy: 0.8216 - val_loss: 0.4113 - val_accuracy: 0.8075
Epoch 6/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.3608 - accuracy: 0.8383 - val_loss: 0.4018 - val_accuracy: 0.8172
Epoch 7/10
1092/1092 [==============================] - 5s 5ms/step - loss: 0.3352 - accuracy: 0.8533 - val_loss: 0.4343 - val_accurac

1092/1092 [==============================] - 7s 7ms/step - loss: 0.3054 - accuracy: 0.8701 - val_loss: 0.4384 - val_accuracy: 0.8060
Epoch 8/10
1092/1092 [==============================] - 7s 7ms/step - loss: 0.2711 - accuracy: 0.8855 - val_loss: 0.4241 - val_accuracy: 0.8096
Epoch 9/10
1092/1092 [==============================] - 7s 7ms/step - loss: 0.2306 - accuracy: 0.9061 - val_loss: 0.4474 - val_accuracy: 0.8144
Epoch 10/10
1092/1092 [==============================] - 7s 7ms/step - loss: 0.1994 - accuracy: 0.9195 - val_loss: 0.5004 - val_accuracy: 0.8137


In [9]:
# import multiprocessing
# p = multiprocessing.Process(target=run_tensorflow)
# p.start()
# p.join()

In [10]:
# model.save("64x3-CNN.model")